In [1]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import heapq #priority queue through a binary heap

In [ ]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])


In [ ]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [40]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

array([[ 6, 10,  8,  7],
       [ 5,  1,  4, 14],
       [ 2, 11,  0, 13],
       [ 3,  9, 12, 15]])

In [ ]:
def is_valid(row, col):
    return (row >= 0) and (row < PUZZLE_DIM) and (col >= 0) and (col < PUZZLE_DIM)

def h_value(state):
    #Approximation Heuristics: Manhattan Distance
    #h = abs (current_cell.x – goal.x) + abs (current_cell.y – goal.y)
    h = 0
    for i in range(PUZZLE_DIM):
        for j in range(PUZZLE_DIM):
            if state[i][j] != 0:
                goal_pos = divmod(state[i][j] - 1, PUZZLE_DIM)
                h += abs(i - goal_pos[0]) + abs(j - goal_pos[1])
    return h

In [ ]:
def a_star_search(initial_state):
    open_list = []
    counter = 0  # to solve ambiguity when comparing tuples
    heapq.heappush(open_list, (0 + h_value(initial_state), 0, counter, initial_state.tobytes(), []))
    closed_set = set()

    evaluated_states = 0
    
    while open_list:
        f, g, _, current_state_bytes, path = heapq.heappop(open_list)
        evaluated_states += 1
        current_state = np.frombuffer(current_state_bytes, dtype=int).reshape((PUZZLE_DIM, PUZZLE_DIM))
        if np.array_equal(current_state, goal_state):
            print("Solution found")
            print(f"Cost (Total number of actions evaluated): {evaluated_states}\n")
            return path + [current_state]
        
        closed_set.add(current_state_bytes)

        for action in available_actions(current_state):
            new_state = do_action(current_state, action)
            new_state_bytes = new_state.tobytes()
            if new_state_bytes not in closed_set:
                heapq.heappush(open_list, (g + 1 + h_value(new_state), g + 1, counter, new_state_bytes, path + [current_state]))
                counter += 1  # counter for each push

    print("No solution found")
    return None
    

In [ ]:
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

path = a_star_search(state)
if path:
    for step, state in enumerate(path):
        print(f"Step {step}:\n{state}\n")
print(f"Total number of steps: {step}\n")
quality = len(path) - 1
print(f"Quality (Number of actions in the solution): {quality}\n")

Solution found
Cost (Total number of actions evaluated): 8653668

Step 0:
[[ 6 10  8  7]
 [ 5  1  4 14]
 [ 2 11  0 13]
 [ 3  9 12 15]]

Step 1:
[[ 6 10  8  7]
 [ 5  1  4 14]
 [ 2 11 13  0]
 [ 3  9 12 15]]

Step 2:
[[ 6 10  8  7]
 [ 5  1  4 14]
 [ 2 11 13 15]
 [ 3  9 12  0]]

Step 3:
[[ 6 10  8  7]
 [ 5  1  4 14]
 [ 2 11 13 15]
 [ 3  9  0 12]]

Step 4:
[[ 6 10  8  7]
 [ 5  1  4 14]
 [ 2 11  0 15]
 [ 3  9 13 12]]

Step 5:
[[ 6 10  8  7]
 [ 5  1  4 14]
 [ 2 11 15  0]
 [ 3  9 13 12]]

Step 6:
[[ 6 10  8  7]
 [ 5  1  4  0]
 [ 2 11 15 14]
 [ 3  9 13 12]]

Step 7:
[[ 6 10  8  7]
 [ 5  1  0  4]
 [ 2 11 15 14]
 [ 3  9 13 12]]

Step 8:
[[ 6 10  0  7]
 [ 5  1  8  4]
 [ 2 11 15 14]
 [ 3  9 13 12]]

Step 9:
[[ 6  0 10  7]
 [ 5  1  8  4]
 [ 2 11 15 14]
 [ 3  9 13 12]]

Step 10:
[[ 6  1 10  7]
 [ 5  0  8  4]
 [ 2 11 15 14]
 [ 3  9 13 12]]

Step 11:
[[ 6  1 10  7]
 [ 5 11  8  4]
 [ 2  0 15 14]
 [ 3  9 13 12]]

Step 12:
[[ 6  1 10  7]
 [ 5 11  8  4]
 [ 0  2 15 14]
 [ 3  9 13 12]]

Step 13:
[[ 6  1 10  